**Импорт библиотек**

In [51]:
import os
import shutil
import json
from PreapreDataLib import create_folder_if_not_exists

from pycocotools.coco import COCO
from mmengine.config import *
from mmengine.runner import set_random_seed
from mmdet.datasets import CocoDataset
from mmdet.apis import init_detector, inference_detector
from mmengine.registry import *
import torch

In [5]:
DATASET_DIR = os.path.abspath('./DatasetCocoFormat/')
TRAIN_DIR = os.path.join(DATASET_DIR, 'train')
TEST_DIR  = os.path.join(DATASET_DIR, 'test')
VAL_DIR = os.path.join(DATASET_DIR, 'val')
ANN_DIR = os.path.join(DATASET_DIR, 'annotations')
TRAIN_ANN_PTH = os.path.join(ANN_DIR, "train.json")
VAL_ANN_PTH = os.path.join(ANN_DIR, "val.json")
TEST_ANN_PTH = os.path.join(ANN_DIR, "test.json")

**Загрузка конфига Mask R-CNN ResNet101 FPN 1x**

In [23]:
CONFIGS_DIR = os.path.abspath("./configs")
create_folder_if_not_exists("./configs")
MRCNN_BASE_CONFIG_PTH = os.path.join(CONFIGS_DIR, "mask-rcnn_r101_fpn_1x_coco.py")
MRCNN_USER_CONFIG_PTH = os.path.join(CONFIGS_DIR, "mask-rcnn_r101_fpn_1x_coco_for_road_damages.py")
if not os.path.exists(MRCNN_BASE_CONFIG_PTH):
    !mim download mmdet --config mask-rcnn_r101_fpn_1x_coco --dest "./configs"

**Изменение конфига под задачу и датасет**